虽然 Mutex和Semaphore 在一定程度上可以互相替代，比如你可以把 值最大为1 的Semaphore当Mutex用，也可以用Mutex＋计数器当Semaphore。但是对于设计理念上还是有不同的，Mutex管理的是资源的使用权，而Semaphore管理的是资源的数量，有那么一点微妙的小区别。打个比方，在早餐餐厅，大家要喝咖啡。如果用Mutex的方式，同时只有一个人可以使用咖啡机，他获得了咖啡机的使用权后，开始做咖啡，其他人只能在旁边等着，直到他做好咖啡后，另外一个人才能获得咖啡机的使用权。如果用Semaphore的模式，服务员会把咖啡做好放到柜台上，谁想喝咖啡就拿走一杯，服务员会不断做咖啡，如果咖啡杯被拿光了，想喝咖啡的人就排队等着。Mutex管理的是咖啡机的使用权，而Semaphore管理的是做好的咖啡数量。


Using Mutex:

A mutex provides mutual exclusion, either producer or consumer can have the key (mutex) and proceed with their work. As long as the buffer is filled by producer, the consumer needs to wait, and vice versa.

At any point of time, only one thread can work with the entire buffer. The concept can be generalized using semaphore.

Using Semaphore:

A semaphore is a generalized mutex. In lieu of single buffer, we can split the 4 KB buffer into four 1 KB buffers (identical resources). A semaphore can be associated with these four buffers. The consumer and producer can work on different buffers at the same time.

最终，GIL保证了MRI中C实现的原生Ruby方法执行的原子性（即便有些警告）。这一行为有时可以帮助作为Ruby开发者的我们，但是GIL其实是为了保护MRI内部而设计的，对Ruby开发者没有可靠的API。

所以GIL不能解决线程安全的问题。就像我说的，使多线程编程正确很难，但是我们每天都在解决棘手的问题。我们面对棘手问题的方法之一是良好的抽象。

In [2]:
class Sheep
  def initialize
    @shorn = false
  end

  def shorn?
    @shorn
  end

  def shear!
    puts "shearing..."
    @shorn = true
  end
end

:shear!

In [3]:
sheep = Sheep.new
sheep_queue = Queue.new
sheep_queue << sheep

5.times.map do
  Thread.new do
      begin
        sheep = sheep_queue.pop(true)

        sheep.shear!
      rescue ThreadError
        # raised by Queue#pop in the threads
        # that don't pop the sheep
      end
  end
end.each(&:join)

shearing...


[#<Thread:0x0000005586135b88@<main>:5 dead>, #<Thread:0x0000005586135980@<main>:5 dead>, #<Thread:0x00000055861273a8@<main>:5 dead>, #<Thread:0x0000005586127268@<main>:5 dead>, #<Thread:0x0000005586127128@<main>:5 dead>]

线程池：一种线程使用模式。线程过多会带来调度开销，进而影响缓存局部性和整体性能。而线程池维护者多个线程，等待着监督管理者分配可并发执行的任务。这避免了在处理短时间任务时创建和销毁线程的代价。线程池不仅能够保证内核的充分利用，还能防止过分调度。可用线程数量应该取决于可用的并发处理、处理器内核、内存、网络`soockets`的数量。例如，线程数一般区cpu数量+2比较合适，线程数过多会导致额外的切换开销

    创建太多线程，将会浪费一定的资源，有些线程未被充分使用。
    销毁太多线程，将导致之后浪费时间再次创建它们。
    创建线程太慢，将会导致长时间的等待，性能变差。
    销毁线程太慢，导致其它线程资源饥饿。


“阻塞”与"非阻塞"与"同步"与“异步"不能简单的从字面理解，提供一个从分布式系统角度的回答。1.同步与异步同步和异步关注的是消息通信机制 (synchronous communication/ asynchronous communication)所谓同步，就是在发出一个*调用*时，在没有得到结果之前，该*调用*就不返回。但是一旦调用返回，就得到返回值了。换句话说，就是由*调用者*主动等待这个*调用*的结果。而异步则是相反，*调用*在发出之后，这个调用就直接返回了，所以没有返回结果。换句话说，当一个异步过程调用发出后，调用者不会立刻得到结果。而是在*调用*发出后，*被调用者*通过状态、通知来通知调用者，或通过回调函数处理这个调用。典型的异步编程模型比如Node.js举个通俗的例子：你打电话问书店老板有没有《分布式系统》这本书，如果是同步通信机制，书店老板会说，你稍等，”我查一下"，然后开始查啊查，等查好了（可能是5秒，也可能是一天）告诉你结果（返回结果）。而异步通信机制，书店老板直接告诉你我查一下啊，查好了打电话给你，然后直接挂电话了（不返回结果）。然后查好了，他会主动打电话给你。在这里老板通过“回电”这种方式来回调。2. 阻塞与非阻塞阻塞和非阻塞关注的是程序在等待调用结果（消息，返回值）时的状态.阻塞调用是指调用结果返回之前，当前线程会被挂起。调用线程只有在得到结果之后才会返回。非阻塞调用指在不能立刻得到结果之前，该调用不会阻塞当前线程。还是上面的例子，你打电话问书店老板有没有《分布式系统》这本书，你如果是阻塞式调用，你会一直把自己“挂起”，直到得到这本书有没有的结果，如果是非阻塞式调用，你不管老板有没有告诉你，你自己先一边去玩了， 当然你也要偶尔过几分钟check一下老板有没有返回结果。在这里阻塞与非阻塞与是否同步异步无关。跟老板通过什么方式回答你结果无关。